<a href="https://colab.research.google.com/github/felipecadm/carteira-de-ativos/blob/main/Aloca%C3%A7%C3%A3o_e_Precifica%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px


In [12]:
acoes = pd.read_csv('acoes.csv')
acoes

,Date,EDP,ITSA,PETR,IBOV
0,2015-02-02,8.795126,6.545879,8.790000,46.270000
1,2015-02-03,8.479606,6.758638,9.970000,47.639999
2,2015-02-04,8.321846,7.006857,10.020000,47.770000
3,2015-02-05,8.183806,7.006857,9.830000,47.880001
4,2015-02-06,8.085206,7.091960,9.120000,47.299999
...,...,...,...,...,...
1886,2022-09-02,22.879999,9.300000,33.419998,107.089996
1887,2022-09-05,22.660000,9.410000,33.340000,108.449997
1888,2022-09-06,22.580000,9.250000,32.099998,105.910004
1889,2022-09-08,22.450001,9.270000,31.799999,106.180000


In [13]:
acoes.set_index('Date', inplace=True)
acoes

,EDP,ITSA,PETR,IBOV
Date,,,,
2015-02-02,8.795126,6.545879,8.790000,46.270000
2015-02-03,8.479606,6.758638,9.970000,47.639999
2015-02-04,8.321846,7.006857,10.020000,47.770000
2015-02-05,8.183806,7.006857,9.830000,47.880001
2015-02-06,8.085206,7.091960,9.120000,47.299999
...,...,...,...,...
2022-09-02,22.879999,9.300000,33.419998,107.089996
2022-09-05,22.660000,9.410000,33.340000,108.449997
2022-09-06,22.580000,9.250000,32.099998,105.910004


In [278]:
Historico_Taxa_Selic = pd.DataFrame(index=[2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022], data=[12.75, 14.25, 12.25, 6.5, 5.0, 2.0, 9.25, 12.35], columns=['Taxa'])
sem_risco = (Historico_Taxa_Selic.mean() / 100)[0]
Historico_Taxa_Selic, sem_risco

(       Taxa
 2015  12.75
 2016  14.25
 2017  12.25
 2018   6.50
 2019   5.00
 2020   2.00
 2021   9.25
 2022  12.35, 0.09293749999999999)

In [273]:
import sys


In [279]:
from pandas.core.describe import describe_categorical_1d
def alocacao_ativos(acoes, investimento, repeticoes, aleatoriedade = 0):
  ativos = acoes.copy()

  melhor_sharpe_ratio = 1 - sys.maxsize
  melhores_pesos = np.empty
  melhor_volatilidade = 0
  melhor_retorno = 0

  if aleatoriedade !=0:
    np.random.seed(aleatoriedade)

  # Normalizar
  for i in ativos.columns:
    acoes[i] = (acoes[i] / acoes[i][0])

  # Aplicar pesos
  for _ in range(repeticoes):
    dataset = ativos.copy()
    pesos = np.random.random(len(dataset.columns[0:4]))
    pesos = pesos / pesos.sum()
  
    for i, acao in enumerate(dataset.columns[0:4]):
      dataset[acao] = dataset[acao] * pesos[i] * investimento

    retorno_carteira = np.log(dataset / dataset.shift(1))
    matriz_covariancia = retorno_carteira[0:4].cov()
    
    dataset['TOTAL'] = dataset.sum(axis=1)
    dataset['TX RETORNO'] = 0.0

    for i in range(1, len(dataset)):
      dataset['TX RETORNO'][i] = ((dataset['TOTAL'][i] / dataset['TOTAL'][i-1]) - 1) * 100

    retorno_esperado = np.sum(dataset['TX RETORNO'].mean() * pesos) * 252
    volatilidade_esperada = np.sqrt(np.dot(pesos, np.dot(matriz_covariancia * 252, pesos)))
    sharpe_ratio = (retorno_esperado - sem_risco) / volatilidade_esperada
    
    if sharpe_ratio > melhor_sharpe_ratio:
      melhor_sharpe_ratio = sharpe_ratio
      melhores_pesos = pesos
      melhor_volatilidade = volatilidade_esperada
      melhor_retorno = retorno_esperado

  balanco_carteira = pd.DataFrame(data = {'Ações': dataset.columns[0:4], 'Pesos': melhores_pesos * 100})

  return dataset, balanco_carteira, melhor_volatilidade, melhor_retorno

In [280]:
acoes_normalizado, balanco_carteira, melhor_volatilidade, melhor_retorno = alocacao_ativos(acoes, 5000, 10, 6)

In [281]:
balanco_carteira

,Ações,Pesos
0,EDP,48.186190
1,ITSA,18.085433
2,PETR,13.038531
3,IBOV,20.689845


In [282]:
acoes_normalizado

,EDP,ITSA,PETR,IBOV,TOTAL,TX RETORNO
Date,,,,,,
2015-02-02,2409.309519,904.271665,651.926547,1034.492269,5000.000000,0.000000
2015-02-03,2322.876871,933.662974,739.443444,1065.122338,5061.105627,1.222113
2015-02-04,2279.660677,967.952857,743.151801,1068.028867,5058.794202,-0.045670
2015-02-05,2241.846540,967.952857,729.060061,1070.488231,5009.347690,-0.977437
2015-02-06,2214.836256,979.709302,676.401599,1057.520706,4928.467863,-1.614578
...,...,...,...,...,...,...
2022-09-02,6267.675985,1284.736061,2478.655756,2394.289437,12425.357239,-0.143125
2022-09-05,6207.410056,1299.931816,2472.722556,2424.695966,12404.760393,-0.165765
2022-09-06,6185.495125,1277.828851,2380.755546,2367.907477,12211.986999,-1.554028


In [283]:
retorno_diário = px.line(x = acoes_normalizado.index, y=acoes_normalizado['TX RETORNO'], title='Retorno Diário 2020 - Atualmente')
retorno_diário.show()

In [284]:
Valorizacao = px.line(title = 'Evolução do Patrimônio')
for ativo in acoes_normalizado.columns[0:4]:
  Valorizacao.add_scatter(x = acoes_normalizado.index, y = acoes_normalizado[ativo], name = ativo)
Valorizacao.show()

In [287]:

def valorizacao_selic(investimento):
  isento_de_ir = investimento
  evolucao = [isento_de_ir]
  for taxa in Historico_Taxa_Selic['Taxa']:
    investimento = investimento + (investimento * (taxa/100))
    evolucao.append(investimento)
  investimento = isento_de_ir + ((investimento - isento_de_ir) * 0.75)
  percentual = ((investimento / isento_de_ir) - 1) * 100
  return investimento, percentual, evolucao

In [294]:
valor_selic, percentual, evolucao_selic = valorizacao_selic(5000)

In [295]:
valor_selic, percentual,  evolucao_selic

(8841.436745723926,
 76.82873491447853,
 [5000,
  5637.5,
  6440.84375,
  7229.8471093749995,
  7699.787171484374,
  8084.7765300585925,
  8246.472060659764,
  9009.270726270792,
  10121.915660965235])

In [296]:
Comparacao = px.line(title="Renda Fixa vs Carteira")
Comparacao.add_scatter(x= acoes_normalizado.index, y=acoes_normalizado['TOTAL'], name="Carteira")
Comparacao.add_scatter(x= Historico_Taxa_Selic.index, y=evolucao_selic, name="Selic")
Comparacao.show()